In [5]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # comment out for interactive
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import FormatStrFormatter
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib.gridspec import GridSpec

# ============================================================
# 1. Load seismicity catalog (3D hypocentres, times, moments)
# ============================================================

seis_df = pd.read_excel(
    "RosemanowesDataPhase2A.xls",
    sheet_name="P only Source Parameters",
    header=4
)

time_all = np.array(seis_df.iloc[:, 1])        # time in hours (since 11/06/82 00:00)
X_all = np.array(seis_df.iloc[:, 4])          # East (m)
Y_all = np.array(seis_df.iloc[:, 5])          # North (m)
Z_all_raw = np.array(seis_df.iloc[:, 6])      # Depth (m), positive down in original
Z_all = -Z_all_raw                            # flip so "down" is negative in plots if needed

pmoment_all = np.array(seis_df.iloc[:, 12])
moment_log10 = np.log10(np.maximum(pmoment_all, 1e10))  # avoid log(0)

# symbol size scaling
size_all = 20 * (moment_log10 - moment_log10.min()) / (
    moment_log10.max() - moment_log10.min() + 1e-9
) + 5

# Move to injection point
r2 = np.loadtxt('rh12.txt')   # RH12 trajectory
r1 = np.loadtxt('rh11.txt')   # RH11 trajectory

# reference injection point = last RH12 point
x_inj = r2[-1, 0]
y_inj = r2[-1, 1]
z_inj = r2[-1, 2]

x_rel = X_all - x_inj
y_rel = Y_all - y_inj
z_rel = -Z_all_raw            # depth positive downward

# well paths relative to injection point
wx = r2[:, 0] - x_inj
wy = r2[:, 1] - y_inj
wz = r2[:, 2]

# production well end relative to injection
wpx = r1[-1, 0] - x_inj
wpy = r1[-1, 1] - y_inj
wpz = z_inj

# rotation by 37.48 degrees
theta = 37.48 * np.pi / 180.0
cos_t = np.cos(theta)
sin_t = np.sin(theta)

def rotate_xy(x, y):
    x0 = x * cos_t - y * sin_t
    y0 = y * cos_t + x * sin_t
    return x0, y0

x0_rel, y0_rel = rotate_xy(x_rel, y_rel)
wx0, wy0 = rotate_xy(wx, wy)
wpx0, wpy0 = rotate_xy(wpx, wpy)

# ============================================================
# 2. Load and process injection/production data
# ============================================================

flow_inj_df = pd.read_excel(
    "RosemanowesDataPhase2A.xls",
    sheet_name="RH12 2A - Flow",
    header=5
)
flow_prod_df = pd.read_excel(
    "RosemanowesDataPhase2A.xls",
    sheet_name="RH11 2A - Flow",
    header=5
)

time_prod = np.array(flow_prod_df.iloc[:, 1])   # hours
rate_prod = np.array(flow_prod_df.iloc[:, 2])   # m3/s or given units

time_inj = np.array(flow_inj_df.iloc[:, 1])     # hours
rate_inj = np.array(flow_inj_df.iloc[:, 2])

# integrate production (cumulative volume)
prod = np.zeros(len(time_prod))
for i in range(1, len(time_prod)):
    dt_h = time_prod[i] - time_prod[i - 1]
    prod[i] = prod[i - 1] + rate_prod[i - 1] * dt_h * 3600.0
prod[-1] = prod[-2]

# integrate injection
inj = np.zeros(len(time_inj))
for i in range(1, len(time_inj)):
    dt_h = time_inj[i] - time_inj[i - 1]
    inj[i] = inj[i - 1] + rate_inj[i - 1] * dt_h * 3600.0
inj[-1] = inj[-2]

# refine time sampling for smooth curves (as in your second script)
s = 3  # samples per hour
t_end = int(time_inj[-1]) + 1
time_fine = np.zeros(t_end * s)
P_fine = np.zeros_like(time_fine)
I_fine = np.zeros_like(time_fine)
Dif_fine = np.zeros_like(time_fine)

for n in range(0, len(time_fine) - 1):
    t = n / s  # hours
    # production interpolation
    if t < time_prod[0]:
        P_fine[n] = 0.0
    elif t > time_prod[-1]:
        P_fine[n] = prod[-1]
    else:
        for i in range(len(time_prod) - 1):
            if time_prod[i] > t:
                P_fine[n] = prod[i - 1] + rate_prod[i - 1] * (t - time_prod[i - 1]) * 3600.0
                break
    # injection interpolation
    if t < time_inj[0]:
        I_fine[n] = 0.0
    elif t > time_inj[-1]:
        I_fine[n] = inj[-1]
    else:
        for i in range(len(time_inj) - 1):
            if time_inj[i] > t:
                I_fine[n] = inj[i - 1] + rate_inj[i - 1] * (t - time_inj[i - 1]) * 3600.0
                break

    time_fine[n] = t
    Dif_fine[n] = I_fine[n] + P_fine[n]

# fill last value
time_fine[-1] = time_fine[-2]
I_fine[-1] = I_fine[-2]
P_fine[-1] = P_fine[-2]
Dif_fine[-1] = Dif_fine[-2]

# ============================================================
# 3. Frame times for movie (use same range as seismic data)
# ============================================================

t_frames = np.arange(3250.0, 8800.0 + 1e-6, 50.0)
n_frames = len(t_frames)

# colormap range for times (same as your vmin/vmax)
t_cmin = 3250
t_cmax = 8800.0

# ============================================================
# 4. Set up combined figure: 2 rows, top row 3 cols, bottom row spans all
# ============================================================

fig = plt.figure(figsize=(12, 11), dpi=200)

# Use GridSpec to control layout
gs = GridSpec(2, 3, figure=fig, height_ratios=[2, 1])  # top row taller

# top row: 3 subplots
ax_xy = fig.add_subplot(gs[0, 0])          # x vs y
ax_xz = fig.add_subplot(gs[0, 1])          # x_rot vs z
ax_yz = fig.add_subplot(gs[0, 2], sharey=ax_xz)  # y_rot vs z

# bottom row: one subplot spanning all 3 columns
ax_vol = fig.add_subplot(gs[1, :])

plt.subplots_adjust(hspace=0.15, wspace=0.10)

# === static setup for seismic panels ===
for ax in (ax_xy, ax_xz, ax_yz):
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)

# XY panel
ax_xy.set_xlabel('x (m): East', fontsize=12)
ax_xy.set_ylabel('y (m): North', fontsize=12)
ax_xy.set_xlim(-700, 700)
ax_xy.set_ylim(-1000, 1000)
ax_xy.set_xticks([-500, 0, 500])
ax_xy.set_yticks([-800, 0, 800])
ax_xy.grid(True, which='both', color='black', linestyle='dashed', alpha=0.3)
ax_xy.set_aspect('equal', adjustable='box')
ax_xy.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax_xy.yaxis.set_major_formatter(FormatStrFormatter('%g'))

# injection well at origin
ax_xy.scatter(0, 0, color='yellow', s=80, marker='s', zorder=5)

# XZ (intermediate axis vs depth)
ax_xz.set_xlabel('x (m): int-axis to NW', fontsize=12)
ax_xz.set_ylabel('z (m): depth', fontsize=12)
ax_xz.set_xlim(-600, 600)
ax_xz.set_ylim(-4000, -1000)
ax_xz.set_xticks([-500, 0, 500])
ax_xz.set_yticks([-4000, -3000, -2000, -1000])
ax_xz.grid(True, which='both', color='black', linestyle='dashed', alpha=0.3)
ax_xz.set_aspect('equal', adjustable='box')
ax_xz.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax_xz.yaxis.set_major_formatter(FormatStrFormatter('%g'))

# plot injection well path
ax_xz.plot(wx0, wz, linewidth=3, color='red')
ax_xz.scatter(wx0[-1], wz[-1], color='yellow', s=80, marker='s', zorder=5)

# YZ (minor axis vs depth)
ax_yz.set_xlabel('x (m): minor-axis to NE', fontsize=12)
ax_yz.set_xlim(-600, 600)
ax_yz.set_ylim(-4000, -1000)
ax_yz.set_xticks([-500, 0, 500])
ax_yz.set_yticks([-4000, -3000, -2000, -1000])
ax_yz.grid(True, which='both', color='black', linestyle='dashed', alpha=0.3)
ax_yz.set_aspect('equal', adjustable='box')
ax_yz.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax_yz.yaxis.set_major_formatter(FormatStrFormatter('%g'))

ax_yz.plot(wy0, wz, linewidth=3, color='red')
ax_yz.scatter(wy0[-1], wz[-1], color='yellow', s=80, marker='s', zorder=5)

# prepare scatter artists (initially empty)
sc_xy = ax_xy.scatter([], [], c=[], s=[], cmap=cm.jet,
                      vmin=t_cmin, vmax=t_cmax, edgecolor='none')
sc_xz = ax_xz.scatter([], [], c=[], s=[], cmap=cm.jet,
                      vmin=t_cmin, vmax=t_cmax, edgecolor='none')
sc_yz = ax_yz.scatter([], [], c=[], s=[], cmap=cm.jet,
                      vmin=t_cmin, vmax=t_cmax, edgecolor='none')

# single colorbar for time
cbar = fig.colorbar(sc_yz, ax=[ax_xy, ax_xz, ax_yz], pad=0.02, fraction=0.03)
cbar.set_label('Time (h)', fontsize=12)
cbar.ax.tick_params(labelsize=10)

title_text = fig.suptitle("", fontsize=16)

# === static setup for volume panel ===
ax_vol.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax_vol.yaxis.set_major_formatter(FormatStrFormatter('%g'))
ax_vol.set_xlabel('Time (h)', fontsize=12)
ax_vol.set_ylabel(r'Fluid Volume ($10^3 \ \mathrm{m}^3$)', fontsize=12)
ax_vol.tick_params(axis='x', labelsize=10)
ax_vol.tick_params(axis='y', labelsize=10)
ax_vol.set_xlim(0, 10000)
ax_vol.set_ylim(-20, 270)

line_prod, = ax_vol.plot([], [], color='cyan', linewidth=2, label='production')
line_inj,  = ax_vol.plot([], [], color='red', linewidth=2, label='injection')
line_net,  = ax_vol.plot([], [], color='green', linewidth=2, label='net injection')

# create legend ONCE and keep a handle
legend = ax_vol.legend(fontsize=10, loc='upper left')

# ============================================================
# 5. Animation functions
# ============================================================

def init():
    sc_xy.set_offsets(np.empty((0, 2)))
    sc_xz.set_offsets(np.empty((0, 2)))
    sc_yz.set_offsets(np.empty((0, 2)))
    sc_xy.set_array(np.array([]))
    sc_xz.set_array(np.array([]))
    sc_yz.set_array(np.array([]))

    line_prod.set_data([], [])
    line_inj.set_data([], [])
    line_net.set_data([], [])

    title_text.set_text("")

    # show legend at the very beginning (frame 0)
    legend.set_visible(True)

    return sc_xy, sc_xz, sc_yz, line_prod, line_inj, line_net, title_text, legend


def update(frame_idx):
    t0 = t_frames[frame_idx]

    # mask events up to this time
    mask = time_all < t0
    x = x_rel[mask]
    y = y_rel[mask]
    z = z_rel[mask]
    t_ev = time_all[mask]
    s_ev = size_all[mask]

    # rotated coordinates for these events
    x0, y0 = rotate_xy(x, y)

    # update scatter plots
    if len(x) > 0:
        off_xy = np.column_stack((x, y))
        off_xz = np.column_stack((x0, z))
        off_yz = np.column_stack((y0, z))
    else:
        off_xy = np.empty((0, 2))
        off_xz = np.empty((0, 2))
        off_yz = np.empty((0, 2))

    sc_xy.set_offsets(off_xy)
    sc_xz.set_offsets(off_xz)
    sc_yz.set_offsets(off_yz)

    sc_xy.set_array(t_ev)
    sc_xz.set_array(t_ev)
    sc_yz.set_array(t_ev)

    sc_xy.set_sizes(s_ev)
    sc_xz.set_sizes(s_ev)
    sc_yz.set_sizes(s_ev)

    # update volume panel: show curves up to t0
    idx = np.where(time_fine <= t0)[0]
    if idx.size > 0:
        k = idx[-1] + 1
        tf = time_fine[:k]
        Pf = -P_fine[:k] / 1e6   # to 10^3 m^3
        If =  I_fine[:k] / 1e6
        Df =  Dif_fine[:k] / 1e6
    else:
        tf = np.array([])
        Pf = np.array([])
        If = np.array([])
        Df = np.array([])

    line_prod.set_data(tf, Pf)
    line_inj.set_data(tf, If)
    line_net.set_data(tf, Df)

    # only show the legend for the first frame (change "< 5" to keep visible longer)
    if frame_idx == 0:
        legend.set_visible(True)
    else:
        legend.set_visible(False)

    title_text.set_text(f"Seismicity and fluid volumes up to t = {int(t0)} h")

    return sc_xy, sc_xz, sc_yz, line_prod, line_inj, line_net, title_text, legend


anim = FuncAnimation(
    fig,
    update,
    init_func=init,
    frames=n_frames,
    interval=300,  # ms between frames
    blit=False
)

# ============================================================
# 6. Save animation
# ============================================================

# MP4 (requires ffmpeg)
# writer = FFMpegWriter(fps=4, bitrate=1800)
# anim.save("seismicity_plus_volumes.mp4", writer=writer)

# Or GIF (requires ImageMagick), comment out MP4 save above:
anim.save("seismicity_plus_volumes.gif", writer='imagemagick', fps=5)

plt.close(fig)


MovieWriter imagemagick unavailable; using Pillow instead.
